In [ ]:
!pip install transformers
!pip install tensorflow
!pip install translate
!pip install evaluate
!pip install torch
!pip install optimum

In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from translate import Translator
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [24]:
#TURKISH DATA IMPORT
df = pd.read_csv('teknofest_train_final.csv', sep='|')

In [25]:
df['labels'] = None
for i in range(len(df)):
    df['labels'][i] = str(str(df['is_offensive'][i])+df['target'][i])

C:\Users\ciglilab\AppData\Local\Temp\ipykernel_2084\3797106863.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['labels'][i] = str(str(df['is_offensive'][i])+df['target'][i])


In [26]:
df['labels'] = df['labels'].map({"1INSULT" : 0,"1RACIST" : 1,"1SEXIST" : 2,"1PROFANITY" : 3,"0OTHER" : 4,"1OTHER" : 5})
df.pop('target')
df.pop('is_offensive')
df.pop('id')
df.head()

,text,labels
0,çürük dişli,0.0
1,Bu adamın islama ve müslümanlara verdiği zarar...,1.0
2,erkekler zora gelmez,2.0
3,Utanmazın götüne kazık sokmuşlar bu tıkırtı ne...,3.0
4,otomasyon< sistemlerine= doğrudan bağlanabilir,4.0


In [27]:
df.labels.value_counts()

4.0    3544
0.0    2407
3.0    2386
2.0    2099
1.0    2054
5.0      72
Name: labels, dtype: int64

In [28]:
test_df = df.tail(int(len(df)*0.2))
df = df.head(int(len(df)*.8))

In [32]:
df['labels'] = df['labels'].astype(int)

In [33]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

In [34]:
from datasets import load_dataset, Dataset

In [35]:
train_dataset = Dataset.from_dict(df)
test_dataset = Dataset.from_dict(test_df)
train_dataset

Dataset({
    features: ['text', 'labels'],
    num_rows: 10093
})

In [36]:
from transformers import AutoTokenizer

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True)
tokenized_test_datasets

Map:   0%|          | 0/10093 [00:00<?, ? examples/s]

Map:   0%|          | 0/2523 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2523
})

In [37]:
#MODEL

from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer, BertForSequenceClassification
import numpy as np
import evaluate